In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import datetime as dt
import numpy as np
from scipy.stats import linregress


# Study data files
cobra_09_path = "data/COBRA-2009-2019.csv"
cobra_20_path = "data/COBRA-2020.csv"
cobra_20_Old_path = "data/COBRA-2020-Old.csv"
cobra_21_path = "data/COBRA-2021.csv"
county_cases_path = "data/county_cases.csv"

# Read the mouse data and the study results
cobra_09 = pd.read_csv(cobra_09_path, low_memory=False)
cobra_20 = pd.read_csv(cobra_20_path)
cobra_20_Old = pd.read_csv(cobra_20_Old_path)
cobra_21 = pd.read_csv(cobra_21_path)
county_cases = pd.read_csv(county_cases_path)


# Combine the data into a single dataset
#study_data_complete = pd.merge(study_results, mouse_metadata, how="left", on="Mouse ID")

# Display the data table for preview
#study_data_complete

In [ ]:
cobra_09.columns = ['offense_id','rpt_date','occur_date','occur_time','poss_date','poss_time','beat','apt_office_prefix', 'apt_office_num', 'location', 'shift_occurence','location_type','UC2_Literal','UC2#','ibr_code','neighborhood','npu','lat','long']
cobra_09 = cobra_09.drop(columns = ['apt_office_prefix','apt_office_num','shift_occurence','location_type','UC2#', 'poss_date'])
cobra_09

In [ ]:
cobra_09['rpt_date']= pd.to_datetime(cobra_09['rpt_date'])
cobra_09['occur_date']= pd.to_datetime(cobra_09['occur_date'])
cobra_09['rpt_date'] = cobra_09['rpt_date'].dt.strftime("%m/%d/%Y")
cobra_09['occur_date'] = cobra_09['occur_date'].dt.strftime("%m/%d/%Y")
cobra_09.head()

In [ ]:
cobra_09.insert(3,'occur_year',cobra_09['occur_date'].str[-4:])
cobra_09_delete = cobra_09.loc[cobra_09['occur_year'].str[:2] != '20']
cobra_09_delete_list = cobra_09_delete['offense_id'].to_list()
cobra_09 = cobra_09[~cobra_09['offense_id'].isin(cobra_09_delete_list)]
cobra_09

In [ ]:
cobra_20_Old.columns = ['offense_id','rpt_date','occur_date','occur_time','poss_date','poss_time','beat','apt_office_prefix', 'apt_office_num', 'location', 'shift_occurence','location_type','UC2_Literal','UC2#','neighborhood','npu','lat','long']
cobra_20_Old = cobra_20_Old.drop(columns = ['apt_office_prefix','apt_office_num','shift_occurence','location_type','UC2#', 'poss_date'])
cobra_20_Old

In [ ]:
cobra_20_Old['rpt_date']= pd.to_datetime(cobra_20_Old['rpt_date'])
cobra_20_Old['occur_date']= pd.to_datetime(cobra_20_Old['occur_date'])
cobra_20_Old['rpt_date'] = cobra_20_Old['rpt_date'].dt.strftime("%m/%d/%Y")
cobra_20_Old['occur_date'] = cobra_20_Old['occur_date'].dt.strftime("%m/%d/%Y")
cobra_20_Old.head()

In [ ]:
cobra_20_Old.insert(3,'occur_year',cobra_20_Old['occur_date'].str[-4:])
cobra_20_Old_delete = cobra_20_Old.loc[cobra_20_Old['occur_year'].str[:2] != '20']
cobra_20_Old_delete_list = cobra_20_Old_delete['offense_id'].to_list()
cobra_20_Old = cobra_20_Old[~cobra_20_Old['offense_id'].isin(cobra_20_Old_delete_list)]
cobra_20_Old

In [ ]:
cobra_20 = cobra_20.drop(columns = ['apt_office_prefix','apt_office_num','MinOfucr','dispo_code','Shift','loc_type', 'poss_date'])
cobra_20_reorder = ['offense_id','rpt_date','occur_date','occur_time','poss_time','beat','location','ibr_code','UC2_Literal','neighborhood','npu','lat','long']
cobra_20_reordered = cobra_20.reindex(columns = cobra_20_reorder)
cobra_20 = cobra_20_reordered.copy()

In [ ]:
cobra_20.insert(3,'occur_year',cobra_20['occur_date'].str[-4:])
cobra_20_delete = cobra_20.loc[cobra_20['occur_year'].str[:2] != '20']
cobra_20_delete_list = cobra_20_delete['offense_id'].to_list()
cobra_20 = cobra_20[~cobra_20['offense_id'].isin(cobra_20_delete_list)]
cobra_20.head()

In [ ]:
cobra_20['rpt_date']= pd.to_datetime(cobra_20['rpt_date'])
cobra_20['occur_date']= pd.to_datetime(cobra_20['occur_date'])
cobra_20['rpt_date'] = cobra_20['rpt_date'].dt.strftime("%m/%d/%Y")
cobra_20['occur_date'] = cobra_20['occur_date'].dt.strftime("%m/%d/%Y")
cobra_20.head()

In [ ]:
cobra_21 = cobra_21.drop(columns = ['occur_day','occur_day_num','occur_day_num','zone', 'poss_date'])
cobra_21.head()

In [ ]:
cobra_21.insert(3,'occur_year',cobra_21['occur_date'].str[-4:])
cobra_21_delete = cobra_21.loc[cobra_21['occur_year'].str[:2] != '20']
cobra_21_delete_list = cobra_21_delete['offense_id'].to_list()
cobra_21 = cobra_21[~cobra_21['offense_id'].isin(cobra_21_delete_list)]
cobra_21

In [ ]:
cobra_21['rpt_date']= pd.to_datetime(cobra_21['rpt_date'])
cobra_21['occur_date']= pd.to_datetime(cobra_21['occur_date'])
cobra_21['rpt_date'] = cobra_21['rpt_date'].dt.strftime("%m/%d/%Y")
cobra_21['occur_date'] = cobra_21['occur_date'].dt.strftime("%m/%d/%Y")
cobra_21.head()

In [ ]:
concat_list = [cobra_09, cobra_20,cobra_20_Old, cobra_21]  # List of your dataframes
crime_df_complete = pd.concat(concat_list)
crime_df_complete.head()

In [ ]:
crime_df_complete.insert(4,'occur_month',crime_df_complete['occur_date'].str[:2])
crime_df_complete.insert(5,'occur_day',crime_df_complete['occur_date'].str[3:5])

In [ ]:
crime_df_complete

In [ ]:
county_cases.columns = ['County Name','Cases','County ID','State FIPS Code','County FIPS Code','Population','Hospitalization','Deaths', 'Case Rate', 'Death Rate', '14 Day Case Rate','14 Day Cases','Antigen Cases','Probable Deaths']
county_cases.head()